In [1]:
#idea taken from https://github.com/damjankuznar/pylof
import scipy.io
from __future__ import division
import warnings
from scipy import stats
import numpy as np
from sklearn import svm
import matplotlib.pyplot as plt
from numbers import Number
from sklearn import preprocessing
import timeit

In [2]:
#load the data
data = scipy.io.loadmat('oc_514.mat')
mdata= data['x']
mdtype = mdata.dtype
ndata = {n: mdata[n][0, 0] for n in mdtype.names}
columns = [n for n, v in ndata.iteritems()]
data =  ndata['data']
data_normalized = preprocessing.normalize(data)


In [ ]:
def distance_euclidean(instance1, instance2):
    def detect_value_type(attribute):
#         """Detects the value type (number or non-number).
#         Returns: (value type, value casted as detected type)"""        
        attribute_type = None
        if isinstance(attribute, Number):
            attribute_type = float
            attribute = float(attribute)
        else:
            attribute_type = str
            attribute = str(attribute)
        return attribute_type, attribute
    # check if instances are of same length
    if len(instance1) != len(instance2):
        raise AttributeError("Instances have different number of arguments.")
    # init differences vector
    differences = [0] * len(instance1)
    # compute difference for each attribute and store it to differences vector
    for i, (attr1, attr2) in enumerate(zip(instance1, instance2)):
        type1, attr1 = detect_value_type(attr1)
        type2, attr2 = detect_value_type(attr2)
        # raise error is attributes are not of same data type.
        if type1 != type2:
            raise AttributeError("Instances have different data types.")
        if type1 is float:
            # compute difference for float
            differences[i] = attr1 - attr2
        else:
            # compute difference for string
            if attr1 == attr2:
                differences[i] = 0
            else:
                differences[i] = 1
    # compute RMSE (root mean squared error)
    rmse = (sum(map(lambda x: x**2, differences)) / len(differences))**0.5
    return rmse

class LOF:
#     instances normalization
    def __init__(self, instances, normalize=True, distance_function=distance_euclidean):
        self.instances = instances
        self.normalize = normalize
        self.distance_function = distance_function
        if normalize:
            self.normalize_instances()

    def attribute_bounds(self):
        maxVal = [float("-inf")] * len(self.instances[0]) 
        minVal = [float("inf")] * len(self.instances[0])
        
        for instance in self.instances:
            minVal = tuple(map(lambda x,y: min(x,y), minVal,instance)) 
            maxVal = tuple(map(lambda x,y: max(x,y), maxVal,instance))

        diff = [dim_max - dim_min for dim_max, dim_min in zip(maxVal, minVal)]
        self.max_attribute_values = maxVal
        self.min_attribute_values = minVal

    def normalize_instances(self):
#         Normalizes the instances and stores the infromation for rescaling new instances.
        if not hasattr(self, "max_attribute_values"):
            self.attribute_bounds()
        new_instances = []
        for instance in self.instances:
            new_instances.append(self.normalize_instance(instance))
        self.instances = new_instances

    def normalize_instance(self, instance):
        return tuple(map(lambda value,max,min: (value-min)/(max-min) if max-min > 0 else 0,
                         instance, self.max_attribute_values, self.min_attribute_values))

    def local_outlier_factor(self, min_pts, instance):      
        if self.normalize:
            instance = self.normalize_instance(instance)
        return local_outlier_factor(min_pts, instance, self.instances, distance_function=self.distance_function)

def k_distance(k, instance, instances, distance_function=distance_euclidean):
#     find the K-nearest neighbors of each point in dataset.
#     Returns: (k-distance, k-distance neighbours)
  
    distances = {}
    for instance2 in instances:
        distance_value = distance_function(instance, instance2)
        if distance_value in distances:
            distances[distance_value].append(instance2)
        else:
            distances[distance_value] = [instance2]
    distances = sorted(distances.items())
    neighbours = []
    [neighbours.extend(n[1]) for n in distances[:k]]
    kd_value = distances[k - 1][0] if len(distances) >= k else distances[-1][0]
    return kd_value, neighbours

def ReachabilityDist(k, instance1, instance2, instances, distance_function=distance_euclidean):
#     """The reachability distance of instance1 with respect to instance2.
#     Returns: reachability distance
#     """
    (kd_value, neighbours) = k_distance(k, instance2, instances, distance_function=distance_function)
    return max([kd_value, distance_function(instance1, instance2)])

def LReachabilityDensity(min_pts, instance, instances, **kwargs):
#     Local reachability density of instance is the inverse of the average reachability
#     distance based on the min_pts-nearest neighbors of instance.
#     Returns: local reachability density
    (kd_value, neighbours) = k_distance(min_pts, instance, instances, **kwargs)
    reachability_arr = [0]*len(neighbours) 
    for i, neighbour in enumerate(neighbours):
        reachability_arr[i] = ReachabilityDist(min_pts, instance, neighbour, instances, **kwargs)
    if not any(reachability_arr):
        warnings.warn("Instance %s (could be normalized) is identical to all the neighbors. Setting local reachability density to inf." % repr(instance))
        return float("inf")
    else:
        return len(neighbours) / sum(reachability_arr)

def local_outlier_factor(min_pts, instance, instances, **kwargs):
#     The (local) outlier factor of instance captures the degree to which we call instance an outlier.
#     Returns: local outlier factor
    (kd_value, neighbours) = k_distance(min_pts, instance, instances, **kwargs)#min_pts is a parameter that is specifying a minimum number of instances to consider for computing LOF value.
    lrd = LReachabilityDensity(min_pts, instance, instances, **kwargs)
    lrd_ratios_array = [0]* len(neighbours)
    for i, neighbour in enumerate(neighbours):
        without_instance = set(instances)
        without_instance.discard(neighbour)
        neighbour_lrd = LReachabilityDensity(min_pts, neighbour, without_instance, **kwargs)
        lrd_ratios_array[i] = neighbour_lrd / lrd
    return sum(lrd_ratios_array) / len(neighbours)

def outliers(k, instances, **kwargs):
    """Identify outliers in the dataset."""
    instances_value_backup = instances
    outliers = []
    for i, instance in enumerate(instances_value_backup):
        print instances_value_backup
        instances = list(instances_value_backup)
        instances.remove(instance)
        l = LOF(instances, **kwargs)
        value = l.local_outlier_factor(k, instance)
        if value > 1:
            outliers.append({"lof": value, "instance": instance, "index": i})
    outliers.sort(key=lambda o: o["lof"], reverse=True)
    return outliers

In [ ]:
instances = data
start = timeit.default_timer()
lof = LOF(instances)

for instance in data:
    value = lof.local_outlier_factor(5, instance)
    print value, instance
stop = timeit.default_timer()
print stop - start 

1.40359432501 [  7.50000000e+01   0.00000000e+00   1.90000000e+02   8.00000000e+01
   9.10000000e+01   1.93000000e+02   3.71000000e+02   1.74000000e+02
   1.21000000e+02  -1.60000000e+01   1.30000000e+01   6.40000000e+01
  -2.00000000e+00   6.30000000e+01   0.00000000e+00   5.20000000e+01
   4.40000000e+01   0.00000000e+00   0.00000000e+00   3.20000000e+01
   0.00000000e+00   0.00000000e+00   0.00000000e+00   0.00000000e+00
   0.00000000e+00   0.00000000e+00   0.00000000e+00   4.40000000e+01
   2.00000000e+01   3.60000000e+01   0.00000000e+00   2.80000000e+01
   0.00000000e+00   0.00000000e+00   0.00000000e+00   0.00000000e+00
   0.00000000e+00   0.00000000e+00   5.20000000e+01   4.00000000e+01
   0.00000000e+00   0.00000000e+00   0.00000000e+00   6.00000000e+01
   0.00000000e+00   0.00000000e+00   0.00000000e+00   0.00000000e+00
   0.00000000e+00   0.00000000e+00   5.20000000e+01   0.00000000e+00
   0.00000000e+00   0.00000000e+00   0.00000000e+00   0.00000000e+00
   0.00000000e+00   